<a href="https://colab.research.google.com/github/jagannath311/DataScience-/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from scipy.linalg import eigh as largest_eigh
from sklearn import svm
from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
from sklearn.metrics import accuracy_score

In [0]:
def get_words_labels(temp_Reviews):
    dict_reviews=[]
    label=[]
    for i in range(0,len(temp_Reviews)-1):
            k=0
            temp=[]
            for j in range(len(temp_Reviews[i])):
                l=temp_Reviews[i][j].split(":")
                if len(l)==2:
                    l[1]=int(l[1])
                    temp.append(l)
            if i+1 <=len(temp_Reviews)-1:
                t=temp_Reviews[i+1][0].split('\n')
                try:
                    temp_Reviews[i+1][0]=t[1]
                except:
                    k+=1
            label.append(num(t[0]))
            dict_reviews.append(temp)
    return dict_reviews,label

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)

In [0]:
def Return_words_in_reviews(path):
    file=open(path)
    txt=file.read()
    Reviews=txt.split('#label#:')
    temp_Reviews=[]
    for i in Reviews:
        temp_Reviews.append(i.split(" "))

    dict_reviews,labels=get_words_labels(temp_Reviews)
    return dict_reviews,labels

In [0]:
def Return_Vocab(Reviews):
    vocab={}
    for i in range(len(Reviews)):
        for j in Reviews[i]:
            if j[0] not in vocab:
                vocab[j[0]]=j[1]
            else:
                vocab[j[0]]+=j[1]
    vocab1=Counter()
    t=list(vocab.keys())
    for i in range(len(t)):
        vocab1[t[i]]+=vocab[t[i]]
    return vocab1

In [0]:
def get_words_dict(Reviews):
    return [dict(i) for i in Reviews]

In [0]:
Book_Reviews,Books_Labels=Return_words_in_reviews('/content/drive/My Drive/processed_stars/electronics/all_balanced.review')

In [0]:
Book_words=get_words_dict(Book_Reviews)

In [0]:
Books_vocab=Return_Vocab(Book_Reviews)

In [0]:
Dvd_Reviews,Dvd_Labels=Return_words_in_reviews('/content/drive/My Drive/processed_stars/kitchen/all_balanced.review')

In [0]:
Dvd_words=get_words_dict(Dvd_Reviews)

In [0]:
Dvd_vocab=Return_Vocab(Dvd_Reviews)

In [0]:
def Domain_Independent_words(vocab1,vocab2,l):
    sort1=vocab1.most_common()
    sort2=vocab2.most_common()
    cnt1=1
    cnt2=cnt(sort1[cnt1][1],sort2)
    while cnt1+cnt2 <= l:
        cnt1+=1
        cnt2=cnt(sort1[cnt1][1],sort2)
    return list(dict(sort1[:cnt1]).keys()),list(dict(sort2[:cnt2]).keys())

In [0]:
def cnt(min_cnt,lis):
    j=0
    try:
        while(lis[j][1]>=min_cnt):
            j+=1
    except:
        return j
    return j+1

In [0]:
def Domain_Specific_words(vocab1,d_indepe):
    return list(set(list(vocab1.keys()))-set(d_indepe))

In [0]:

Books_domain_indep_B_D,Dvd_domain_indep_B_D=Domain_Independent_words(Books_vocab,Dvd_vocab,782)
print(len(list(set(Books_domain_indep_B_D)|set(Dvd_domain_indep_B_D))))

500


In [0]:
Domain_Independent_BD=list(set(Books_domain_indep_B_D)|set(Dvd_domain_indep_B_D))

In [0]:
Books_vocab1=dict(Books_vocab.most_common(11949))
Dvd_vocab1=dict(Dvd_vocab.most_common(9197))

In [0]:
Book_domain_dep_B_D=Domain_Specific_words(Books_vocab1,Domain_Independent_BD)
Dvd_domain_dep_B_D=Domain_Specific_words(Dvd_vocab1,Domain_Independent_BD)

In [0]:
Domain_Specific_B_D=list(set(Book_domain_dep_B_D)|set(Dvd_domain_dep_B_D))

In [0]:
    cnt=0
    if word1 in Review1 and word2 in Review1:
        cnt += min(Review1[word1],Review1[word2])
    if word1 in Review2 and word2 in Review2:
        cnt += min(Review2[word1],Review2[word2])
    return cnt

NameError: ignored

In [0]:
co_occurance=[]
for i in range(len(Domain_Specific_B_D)):
    temp=[]
    for j in range(0,len(Domain_Independent_BD)):
        k=occurance(Domain_Specific_B_D[i],Domain_Independent_BD[j],Books_vocab,Dvd_vocab)
        temp.append(k)
    co_occurance.append(temp)

NameError: ignored

In [0]:
M=np.asarray(co_occurance)

In [0]:
def AffinityMatrix(M):
    m=M.shape[0]+M.shape[1]
    l=M.shape[1]
    temp=np.zeros((m,m))
    M1=np.transpose(M)
    for i in range(0,m-l):
        for j in range(m):
            if j<m-l:
                temp[i][j]=0
            else:
                temp[i][j]=M[i][j-m+l]
    for i in range(m-l,m):
        for j in range(m):
            if j>=m-l:
                temp[i][j]=0
            else:
                temp[i][j]=M1[i-m+l][j]
    return temp    

In [0]:
Aff=AffinityMatrix(M)

In [0]:
D=np.zeros((len(Aff),len(Aff)))
for i in range(len(Aff)):
    D[i][i]=sum(Aff[i][j] for j in range(len(Aff[i])))
D_inv=np.linalg.matrix_power(D,-1)
Laplacian=np.matmul(np.matmul(D_inv,Aff),D_inv)

In [0]:
k=10
evals_large_sparse, evecs_large_sparse = largest_eigsh(Laplacian, k, which='LM')

In [0]:
def Return_Features_For_mapping(words,Domain_Specific):
    l=[]
    z=0
    for i in range(len(words)):
        temp=np.zeros(len(Domain_Specific))
        for word in words[i].keys():
            try:
                ind=Domain_Specific.index(word)
                temp[ind]+=1
            except:
                z+=1
        l.append(temp)
    return l

In [0]:
def general_features(words,Domain_specific,Domain_indepe):
    comb_vocab=Domain_specific.copy()
    for j in Domain_indepe:
        comb_vocab.append(j)
    z=0
    l=[]
    for i in range(len(words)):
        temp=np.zeros(len(comb_vocab))
        for word in words[i].keys():
            try:
                ind=comb_vocab.index(word)
                temp[ind]+=1
            except:
                z=1
        l.append(temp)

    return l

In [0]:
def Whole_Features(features,extra_features):
    l=[]
    for i,j in zip(features,extra_features):
        temp=list(i.copy())
        for s in j:
            temp.append(s)
        l.append(temp)
    return l

In [0]:
def Extract_Features(features_for_Mapping,gamma,U):
    l=[]
    for f1 in features_for_Mapping:
        s=[]
        t=list(gamma*np.matmul(f1,U))
        for i in t:
            s.append(i)
        l.append(s)
    return l

In [0]:
Books_general_features=general_features(Book_words,Domain_Specific_B_D,Domain_Independent_BD)
Books_mapping_features=Return_Features_For_mapping(Book_words,Domain_Specific_B_D)

In [0]:
Books_Extra_Features=Extract_Features(Books_mapping_features,0.6,evecs_large_sparse[:len(Domain_Specific_B_D)])
Books_whole_features=Whole_Features(Books_general_features,Books_Extra_Features)

In [0]:
Dvd_general_features=general_features(Dvd_words,Domain_Specific_B_D,Domain_Independent_BD)
Dvd_mapping_features=Return_Features_For_mapping(Dvd_words,Domain_Specific_B_D)

In [0]:
Dvd_Extra_Features=Extract_Features(Dvd_mapping_features,0.6,evecs_large_sparse[:len(Domain_Specific_B_D)])
Dvd_whole_features=Whole_Features(Dvd_general_features,Dvd_Extra_Features)

In [0]:
Whole_BD_Features=Books_whole_features.copy()
for j in Dvd_whole_features:
    Whole_BD_Features.append(j)

In [0]:
def Binarise_Labels(Labels):
    bin_labels=[]
    for i in Labels:
        if i>=3:
            bin_labels.append(1)
        else:
            bin_labels.append(0)
    return bin_labels

In [0]:
Books_Binarise_Labels=Binarise_Labels(Books_Labels)
Dvd_Binarise_Labels=Binarise_Labels(Dvd_Labels)

In [0]:
BD_Whole_labels=Books_Binarise_Labels.copy()
for i in Dvd_Binarise_Labels:
    BD_Whole_labels.append(i)

In [0]:
x1_train,x1_test,y1_train,y1_test=train_test_split(Whole_BD_Features,BD_Whole_labels,test_size=0.3,random_state=10,stratify=BD_Whole_labels)

In [0]:
#-------------------------------------------------------------------BUILDING SVM CLASSIFIER MODEL FOR UNIGRAM FREQUENCY FEATURES-------------------------------------------------------------------------------
#--------------------------takes a couple of minutes--------------------------------------------
svm_clf=svm.SVC()
svm_clf.fit(x1_train,y1_train)
y1_pred_svm=svm_clf.predict(x1_test)
print("Testing data Accuracy Score of SVM Classifier Model: ",accuracy_score(y1_test,y1_pred_svm))

In [0]:
nb1=GaussianNB()
nb1.fit(x1_train,y1_train)
print('Training Accuracy Score of Naive Bayes Model: ',nb1.score(x1_train,y1_train))
y1_predict=nb1.predict(x1_test)
print('Testing Accuracy Score of Naive Bayes Model:  ',accuracy_score(y1_test,y1_predict))

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x1_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
x1_train = scaler.transform(x1_train)
x1_test = scaler.transform(x1_test)

In [0]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=2)

In [0]:
mlp.fit(x1_train,y1_train)
predictions = mlp.predict(x1_test)
